In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse

p = Path('.')

json_string='''@article{womack1996brokerage,
  title={Do brokerage analysts' recommendations have investment value?},
  author={Womack, Kent L},
  journal={The journal of finance},
  volume={51},
  number={1},
  pages={137--167},
  year={1996},
  publisher={Wiley Online Library}
}'''

url= f'https://scholar.google.com/scholar?cites=13730656698353056628&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')

!clear
!git status
!git add .
!git commit -m '$citation_key'
!git push

citation_dict

article--1996--Sloan-Richard-G--Do-stock-prices-fully-reflect-information-in-accruals-and-cash-flows-about-future-earnings--Accounting-review.pdf
CITATION KEY:  sloan1996stock
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
Everything up-to-date


{'citation_key': 'sloan1996stock',
 'google_scholar_link': 'https://scholar.google.com/scholar?cites=7509362089093863533&as_sdt=5,47&sciodt=0,47&hl=en',
 'cited_by': [{'article_title': 'The effects of financial reporting and disclosure on corporate investment: A review',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S0165410119300412',
   'cited_by': '/scholar?cites=13786672447483382540&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '676',
   'auhtors': ['S Roychowdhury', 'N Shroff', 'RS Verdi']},
  {'article_title': 'Understanding earnings quality: A review of the proxies, their determinants and their consequences',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S0165410110000339',
   'cited_by': '/scholar?cites=7545076372643935259&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '5826',
   'auhtors': ['P Dechow', 'W Ge', 'C Schrand']},
  {'article_title': 'Responsible investing: The ESG-efficient frontier',
   'article_link':